In [1]:
import pandas as pd
import numpy as np
import torch
import pickle

### Process the 48*N-dimensional matrix related to genes extracted from case samples :multiomics_features.tsv
`'Path'` is the path of the. csv file where ppi relationships are stored

`feature_Path` is the path of all the features extracted from patients

In [2]:
Path = '../data/ppi/CPDB_genename_edgelist_0.5.tsv'
feature_Path = '../data/biological_features.csv'

In [3]:
cpdb_genename = pd.read_csv(Path,header=0,
                           sep='\t',
                           encoding='utf8') 
extract_feature= pd.read_csv(feature_Path,header=0,
                           sep='\t',
                           encoding='utf8') 

In [4]:
extract  = extract_feature.values.tolist()
extract_names= [row[0] for row in extract]

In [5]:
lst = cpdb_genename.values
lst = lst.tolist()

In [6]:
print(lst[0:5])

[['STIM1', 'TRPC1', 0.5971769999999998], ['ATP2B4', 'NOS1', 0.5208359999999999], ['ABCC9', 'KCNJ11', 0.990732], ['HADHB', 'HADHA', 0.99998], ['GTF2E1', 'GTF2E2', 0.97317]]


In [7]:
origin_list =[]
new_list = []
have_neighbor = set()
for row in lst:
    g1 = row[0]
    g2 = row[1]
    if g1 in extract_names and g2 in extract_names:
        origin_list.append((g1,g2))
        new_list.append((g1,g2))
        new_list.append((g2,g1))
        have_neighbor.add(g1)
        have_neighbor.add(g2)

In [8]:
print(len(new_list))

628064


In [12]:
gene_save_path = '../data/step1/'
gene_feat = {}
for row in extract_feature.itertuples():
    #print(index)
    str1 = row[1]
    if (str1 in have_neighbor):
        mylist = row[2:]
        mytensor = torch.Tensor(mylist)
        gene_feat[str1] = mytensor
gene_name_ids = {}
gene_names = list(gene_feat.keys())
for i in range(len(gene_names)):
    gene_name_ids[gene_names[i]]  = i
with open(gene_save_path+'gene_toids.pkl','wb')as f:
    pickle.dump(gene_name_ids,f)

In [13]:
b = list(set(new_list))
print(len(b))
edges = []
for edge in b:
    x = gene_name_ids[edge[0]]
    y = gene_name_ids[edge[1]]
    edges.append((x,y))

627420


In [14]:
origin_relations = []
for edge in origin_list:
    x = gene_name_ids[edge[0]]
    y = gene_name_ids[edge[1]]
    origin_relations.append((x,y))

In [15]:
with open(gene_save_path+'gene_relations_undirected.pkl','wb')as f:
    pickle.dump(edges,f)
with open(gene_save_path+'gene_relations_directed.pkl','wb')as f:
    pickle.dump(origin_relations,f)

#### Let the protein interaction network be completely composed of gene nodes with known characteristics, and update the characteristic matrix of gene nodes at the same time。

In [17]:
with open(gene_save_path+'gene_feature_bio.pkl','wb')as f:
    pickle.dump(gene_feat,f)

#### generate homogeneous network for test 

In [6]:
import dgl
import pickle
import torch

In [2]:
gene_ids_file = '../data/step1/gene_toids.pkl'
gene_feat_file = '../data/step1/gene_feature_bio.pkl'
grapth1_file = '../data/step1/gene_relations_directed.pkl'
grapth2_file = '../data/step1/gene_relations_undirected.pkl'
with open(gene_ids_file,'rb')as f:
    gene_ids = pickle.load(f)
with open(gene_feat_file,'rb')as f:
    gene_feat_new = pickle.load(f)
with open(grapth1_file,'rb')as f:
    directed_relations = pickle.load(f)
with open(grapth2_file,'rb')as f:
    undirected_relations = pickle.load(f)

In [4]:
g1,g2 = [e[0] for e in directed_relations],[e[1] for e in directed_relations] 
g_1,g_2 = [e[0] for e in undirected_relations],[e[1] for e in undirected_relations] 

In [7]:
tensors1 = [t for t in gene_feat_new.values()]
gene_feat_tensor = torch.stack(tensors1)

In [12]:
print(gene_feat_tensor)

tensor([[0.0000, 0.0053, 0.0000,  ..., 0.0304, 0.0910, 0.0173],
        [0.0000, 0.0053, 0.0000,  ..., 0.0662, 0.3309, 0.0425],
        [0.0633, 0.0158, 0.0557,  ..., 0.2257, 0.3389, 0.5127],
        ...,
        [0.0000, 0.0106, 0.0000,  ..., 0.0502, 0.0153, 0.0433],
        [0.0000, 0.0159, 0.0000,  ..., 0.1889, 0.2675, 0.0800],
        [0.0628, 0.0619, 0.1089,  ..., 0.0416, 0.3418, 0.2629]])


In [8]:
mygraph1 = dgl.heterograph({
            ('Gene','gg','Gene'):(g1,g2)
        })
mygraph1.nodes['Gene'].data['feature'] = gene_feat_tensor

{'feature': tensor([[0.0000, 0.0053, 0.0000,  ..., 0.0304, 0.0910, 0.0173],
        [0.0000, 0.0053, 0.0000,  ..., 0.0662, 0.3309, 0.0425],
        [0.0633, 0.0158, 0.0557,  ..., 0.2257, 0.3389, 0.5127],
        ...,
        [0.0000, 0.0106, 0.0000,  ..., 0.0502, 0.0153, 0.0433],
        [0.0000, 0.0159, 0.0000,  ..., 0.1889, 0.2675, 0.0800],
        [0.0628, 0.0619, 0.1089,  ..., 0.0416, 0.3418, 0.2629]])}


In [23]:
print(mygraph1.ndata)
print(mygraph1.nodes['Gene'])

{'feature': tensor([[0.0000, 0.0053, 0.0000,  ..., 0.0304, 0.0910, 0.0173],
        [0.0000, 0.0053, 0.0000,  ..., 0.0662, 0.3309, 0.0425],
        [0.0633, 0.0158, 0.0557,  ..., 0.2257, 0.3389, 0.5127],
        ...,
        [0.0000, 0.0106, 0.0000,  ..., 0.0502, 0.0153, 0.0433],
        [0.0000, 0.0159, 0.0000,  ..., 0.1889, 0.2675, 0.0800],
        [0.0628, 0.0619, 0.1089,  ..., 0.0416, 0.3418, 0.2629]])}
NodeSpace(data={'feature': tensor([[0.0000, 0.0053, 0.0000,  ..., 0.0304, 0.0910, 0.0173],
        [0.0000, 0.0053, 0.0000,  ..., 0.0662, 0.3309, 0.0425],
        [0.0633, 0.0158, 0.0557,  ..., 0.2257, 0.3389, 0.5127],
        ...,
        [0.0000, 0.0106, 0.0000,  ..., 0.0502, 0.0153, 0.0433],
        [0.0000, 0.0159, 0.0000,  ..., 0.1889, 0.2675, 0.0800],
        [0.0628, 0.0619, 0.1089,  ..., 0.0416, 0.3418, 0.2629]])})


In [33]:
save_path1 = '../data/network/homo/directed_graph_newfeat.bin'
dgl.save_graphs(save_path1, [mygraph1])

In [34]:
mygraph2 = dgl.heterograph({
            ('Gene','gg','Gene'):(g_1,g_2)
        })
mygraph2.nodes['Gene'].data['feature'] = gene_feat_tensor

In [35]:
save_path2 = '../data/network/homo/undirected_graph_newfeat.bin'
dgl.save_graphs(save_path2, [mygraph2])